In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    #recurrent_training_input = data['recurrent_training_input']
    #prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    action_testing_input = data['action_testing_input']
    #recurrent_testing_input = data['recurrent_testing_input']
    #prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    action_model = Dropout(rate=0.5)(action_input_layer)
    action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    #recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    #recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    #recurrent_model = Dropout(rate=0.5)(recurrent_model)
    #recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    #prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    #prior_model = Dropout(rate=0.5)(prior_input_layer)
    #prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    model = Concatenate()([action_model])
    model = Dropout(rate=0.5)(action_model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model(action_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'action': action_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'action': action_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('actions_cross_validation_results.csv', index=False)

Epoch 1/1000
162/162 [==============================] - 18s 98ms/step - loss: 786.7687 - completion_loss: 18.7387 - problems_loss: 768.0300 - val_loss: 32.2143 - val_completion_loss: 10.4377 - val_problems_loss: 21.7766
Epoch 2/1000
162/162 [==============================] - 16s 98ms/step - loss: 52.9988 - completion_loss: 9.8788 - problems_loss: 43.1200 - val_loss: 31.1865 - val_completion_loss: 10.1599 - val_problems_loss: 21.0266
Epoch 3/1000
162/162 [==============================] - 16s 98ms/step - loss: 51.6727 - completion_loss: 9.5504 - problems_loss: 42.1223 - val_loss: 30.2060 - val_completion_loss: 10.0044 - val_problems_loss: 20.2016
Epoch 4/1000
162/162 [==============================] - 17s 108ms/step - loss: 50.2458 - completion_loss: 9.3519 - problems_loss: 40.8939 - val_loss: 29.1999 - val_completion_loss: 9.9238 - val_problems_loss: 19.2761
Epoch 5/1000
162/162 [==============================] - 18s 114ms/step - loss: 48.7781 - completion_loss: 9.2384 - problems_loss:

Epoch 39/1000
162/162 [==============================] - 11s 68ms/step - loss: 34.0061 - completion_loss: 9.0601 - problems_loss: 24.9460 - val_loss: 17.0584 - val_completion_loss: 9.8959 - val_problems_loss: 7.1624
Epoch 40/1000
162/162 [==============================] - 14s 86ms/step - loss: 34.1215 - completion_loss: 9.0230 - problems_loss: 25.0985 - val_loss: 17.0528 - val_completion_loss: 9.9142 - val_problems_loss: 7.1386
Epoch 41/1000
162/162 [==============================] - 15s 92ms/step - loss: 34.0356 - completion_loss: 9.0153 - problems_loss: 25.0203 - val_loss: 17.0593 - val_completion_loss: 9.8795 - val_problems_loss: 7.1798
Epoch 42/1000
162/162 [==============================] - 16s 99ms/step - loss: 33.9221 - completion_loss: 9.0943 - problems_loss: 24.8278 - val_loss: 17.0939 - val_completion_loss: 9.8933 - val_problems_loss: 7.2006
Epoch 43/1000
162/162 [==============================] - 15s 93ms/step - loss: 33.7424 - completion_loss: 9.0554 - problems_loss: 24.687

Epoch 33/1000
162/162 [==============================] - 11s 70ms/step - loss: 37.3167 - completion_loss: 9.2368 - problems_loss: 28.0799 - val_loss: 18.4761 - val_completion_loss: 10.1325 - val_problems_loss: 8.3435
Epoch 34/1000
162/162 [==============================] - 10s 59ms/step - loss: 37.2234 - completion_loss: 9.2647 - problems_loss: 27.9588 - val_loss: 18.3870 - val_completion_loss: 10.1308 - val_problems_loss: 8.2563
Epoch 35/1000
162/162 [==============================] - 10s 64ms/step - loss: 37.0096 - completion_loss: 9.2655 - problems_loss: 27.7441 - val_loss: 18.3104 - val_completion_loss: 10.1338 - val_problems_loss: 8.1766
Epoch 36/1000
162/162 [==============================] - 10s 64ms/step - loss: 37.0415 - completion_loss: 9.2526 - problems_loss: 27.7889 - val_loss: 18.2448 - val_completion_loss: 10.1338 - val_problems_loss: 8.1110
Epoch 37/1000
162/162 [==============================] - 11s 66ms/step - loss: 36.8123 - completion_loss: 9.2594 - problems_loss: 27

Epoch 71/1000
162/162 [==============================] - 9s 57ms/step - loss: 35.3860 - completion_loss: 9.1505 - problems_loss: 26.2355 - val_loss: 17.8342 - val_completion_loss: 9.9892 - val_problems_loss: 7.8450
Epoch 72/1000
162/162 [==============================] - 8s 51ms/step - loss: 35.2523 - completion_loss: 9.1257 - problems_loss: 26.1266 - val_loss: 17.8441 - val_completion_loss: 9.9859 - val_problems_loss: 7.8582
Epoch 73/1000
162/162 [==============================] - 12s 73ms/step - loss: 35.3686 - completion_loss: 9.1419 - problems_loss: 26.2267 - val_loss: 17.8390 - val_completion_loss: 9.9784 - val_problems_loss: 7.8606
Epoch 74/1000
162/162 [==============================] - 13s 80ms/step - loss: 35.4339 - completion_loss: 9.1387 - problems_loss: 26.2951 - val_loss: 17.7937 - val_completion_loss: 9.9847 - val_problems_loss: 7.8090
Epoch 75/1000
162/162 [==============================] - 12s 74ms/step - loss: 35.2651 - completion_loss: 9.1456 - problems_loss: 26.1195 

Epoch 34/1000
162/162 [==============================] - 11s 66ms/step - loss: 17.6820 - completion_loss: 9.3611 - problems_loss: 8.3209 - val_loss: 17.4288 - val_completion_loss: 10.0403 - val_problems_loss: 7.3885
Epoch 35/1000
162/162 [==============================] - 11s 66ms/step - loss: 17.6900 - completion_loss: 9.3830 - problems_loss: 8.3070 - val_loss: 17.1150 - val_completion_loss: 10.0434 - val_problems_loss: 7.0716
Epoch 36/1000
162/162 [==============================] - 10s 63ms/step - loss: 17.6582 - completion_loss: 9.3655 - problems_loss: 8.2928 - val_loss: 17.0828 - val_completion_loss: 10.0415 - val_problems_loss: 7.0413
Epoch 37/1000
162/162 [==============================] - 12s 74ms/step - loss: 17.5955 - completion_loss: 9.3583 - problems_loss: 8.2372 - val_loss: 17.0551 - val_completion_loss: 10.0548 - val_problems_loss: 7.0003
Epoch 38/1000
162/162 [==============================] - 11s 66ms/step - loss: 17.5934 - completion_loss: 9.3652 - problems_loss: 8.2282

Epoch 11/1000
162/162 [==============================] - 11s 66ms/step - loss: 35.4423 - completion_loss: 9.5133 - problems_loss: 25.9291 - val_loss: 19.9765 - val_completion_loss: 9.4531 - val_problems_loss: 10.5235
Epoch 12/1000
162/162 [==============================] - 12s 77ms/step - loss: 35.3945 - completion_loss: 9.4832 - problems_loss: 25.9114 - val_loss: 19.5916 - val_completion_loss: 9.4652 - val_problems_loss: 10.1264
Epoch 13/1000
162/162 [==============================] - 12s 71ms/step - loss: 35.1070 - completion_loss: 9.4937 - problems_loss: 25.6133 - val_loss: 19.6406 - val_completion_loss: 9.4797 - val_problems_loss: 10.1610
Epoch 14/1000
162/162 [==============================] - 11s 68ms/step - loss: 35.3134 - completion_loss: 9.4755 - problems_loss: 25.8378 - val_loss: 19.5851 - val_completion_loss: 9.4641 - val_problems_loss: 10.1210
Epoch 15/1000
162/162 [==============================] - 10s 62ms/step - loss: 35.0926 - completion_loss: 9.4547 - problems_loss: 25

Epoch 49/1000
162/162 [==============================] - 12s 72ms/step - loss: 34.6664 - completion_loss: 9.3722 - problems_loss: 25.2941 - val_loss: 19.5455 - val_completion_loss: 9.3008 - val_problems_loss: 10.2447
Epoch 50/1000
162/162 [==============================] - 12s 72ms/step - loss: 34.6247 - completion_loss: 9.3232 - problems_loss: 25.3015 - val_loss: 19.6217 - val_completion_loss: 9.2804 - val_problems_loss: 10.3414
Epoch 51/1000
162/162 [==============================] - 11s 69ms/step - loss: 34.7922 - completion_loss: 9.3108 - problems_loss: 25.4814 - val_loss: 19.4003 - val_completion_loss: 9.2638 - val_problems_loss: 10.1365
Epoch 52/1000
162/162 [==============================] - 11s 67ms/step - loss: 34.7878 - completion_loss: 9.2877 - problems_loss: 25.5001 - val_loss: 19.3852 - val_completion_loss: 9.2529 - val_problems_loss: 10.1323
Epoch 53/1000
162/162 [==============================] - 11s 68ms/step - loss: 34.3695 - completion_loss: 9.3261 - problems_loss: 25

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('actions_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []

# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('actions_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
26,fold_0,None,1722,0.575102,0.683508,-0.004358,0.009432,0.217268,2.505995,0.155909,-0.032503,-0.005681,16.162571
27,fold_1,None,1722,0.590310,0.725319,-0.002464,-0.002464,0.197602,2.061050,0.297166,0.019961,0.039453,7.294046
28,fold_2,None,1721,0.616113,0.749564,-0.006630,0.005925,0.188962,3.026193,0.063625,-0.037749,-0.002676,84.249469
29,fold_3,None,1721,0.546573,0.726322,0.000901,0.002274,0.198599,2.221199,0.217191,-0.001406,0.000304,11.998868
30,fold_4,None,1721,0.600434,0.719349,0.020526,0.021888,0.197991,2.046128,0.232629,0.008902,0.009336,8.432688
31,all_target_data,None,8607,0.552146,0.720809,0.003970,0.004116,0.200086,2.375084,0.192943,-0.018993,-0.012346,26.131637
